### 1. PDF 문서 이미지 추출

In [ ]:
%pip install unstructured==0.12.5 unstructured-inference
%pip install pdfminer.six==20231228
%pip install pdf2image pillow-heif opencv-python
%pip install llama-index-vector-stores-qdrant

In [ ]:
# 이미지 문서 추출
from unstructured.partition.pdf import partition_pdf
image_save_path = "./data_image/pdf2image" # 추출 할 이미지 저장 경로
fpath = "./data/2005.11401v4.pdf" # 사용 할 논문 파일 경로
'''
unstructured를 사용하여 pdf 이미지만 파일로 추출하기
'''
raw_pdf_elements = partition_pdf(
    filename=fpath,
    extract_images_in_pdf=True, # 이미지 추출
    extract_image_block_output_dir=image_save_path,
    strategy="fast",              # pdfminer / 기본 파서만 사용
    infer_table_structure=False   # 표 구조 분석 모델 호출 차단
)


In [ ]:
# 이미지를 출력하는 함수 작성
from PIL import Image
import matplotlib.pyplot as plt

def plot_images(image_paths):
    images_shown = 0
    plt.figure(figsize=(16, 9))
    for img_path in image_paths:
        img_path = img_path.lower() # 소문자화
        if ".jpg" in img_path:
            image = Image.open(img_path)

            plt.subplot(3, 3, images_shown + 1)
            plt.imshow(image)
            plt.xticks([])
            plt.yticks([])

            images_shown += 1
            if images_shown >= 9:
                break

In [ ]:
# 추출 된 이미지 출력해보기
import os
image_save_path = "./data_image/pdf2image" # 추출 할 이미지 저장 경로
file_list = [os.path.join(image_save_path, f) for f in os.listdir(image_save_path)]
plot_images(file_list) # 추출 된 이미지 plot

### 2. PDF 문서 테이블, 텍스트 등 파서

In [ ]:
# 이미지를 문서로 읽어오기
from llama_index.core import SimpleDirectoryReader
image_save_path = "./data_image/pdf2image" # 이미지가 저장 된 경로
image_documents = SimpleDirectoryReader(input_dir=image_save_path).load_data()
print(f"이미지 문서 개수 : {len(image_documents)}")


In [ ]:
# 저장 된 이미지 타입 확인
for doc in image_documents:
    print(type(doc))

In [ ]:
# Step1. .evn 파일 로드 및 비동기 문제 해결
from dotenv import load_dotenv 
import nest_asyncio
load_dotenv() # 환경 변수 설정
nest_asyncio.apply() # 비동기 처리 문제 해결

In [ ]:
# Step2. 라마파스 사용하여 문서 로드하기
from llama_parse import LlamaParse
parser = LlamaParse(result_type="text") # markdown 혹은 text 사용 가능
file_extractor = {".pdf": parser}  # PDF 파일을 LlamaParse로 처리하도록 설정
fpath = "./data/2005.11401v4.pdf" # 사용 할 논문 파일 경로
text_documents = SimpleDirectoryReader(
    input_files=[fpath], # 논문 저장경로
    file_extractor=file_extractor
).load_data()

In [ ]:
print(f"논문 TEXT 문서 개수 : {len(text_documents)}")

### 3. 이미지 캡셔닝

ImageDocuments는 llama-index 0.14.2(25년9월 기준) 버전 기준으로 멀티모달 LLM 호출에 전달할 때, 사용불가
ImageNode로 사용하는 것으로 권장

In [ ]:
# Step1. ImageDocument를 ImageNode로 변환

from llama_index.core.schema import ImageDocument, ImageNode 
from typing import List

# 이미지 문서 -> 이미지 노드 변환함수 작성(이미지 경로 포함)
def docs_to_image_nodes(img_docs: List[ImageDocument]) -> List[ImageNode]:
    nodes = []
    for d in img_docs:
        # 우선순위: d.image_path -> d.metadata["file_path"] -> None
        path = getattr(d, "image_path", None) or d.metadata.get("file_path")
        node = ImageNode(
            image_path=path,                # ✅ 경로를 여기에
            image_url=getattr(d, "image_url", None),
            image=getattr(d, "image", None),  # base64일 수 있음
            metadata=dict(d.metadata),     # 기존 메타데이터 유지
            text=getattr(d, "text", None),
        )
        nodes.append(node)
    return nodes

In [ ]:
# Step2. 이미지 요약을 생성하고 캡션을 추가하는 함수 작성

from llama_index.multi_modal_llms.openai import OpenAIMultiModal

# 이미지 캡셔닝 함수 정의
def summarize_images_captioning(image_nodes: list[ImageNode]) -> List[ImageNode]:
    # 멀티모달 모델 인스턴스화
    openai_mm_llm = OpenAIMultiModal(
        model='gpt-4o',         # 실제 사용하는 모델명으로 변경
        max_new_tokens=3000,
        temperature=0.5
    )
    
    prompt_template = """
    당신은 주어진 이미지를 텍스트로 설명을 바꾸는 역할을 해야 합니다.
    이미지에 대한 상세 설명을 해주세요.
    해당 이미지에 텍스트나 숫자, 수식 등이 있는 경우 해당 용어를 반드시 모두 사용해서 어떤 그림인지 설명해주세요.
    그래프나 차트의 경우도 숫자, 수식 등의 데이터를 포함하여 상세 설명해주시길 바랍니다.
    """
    
    for img_node in image_nodes:
        # 이미지를 단건으로 넣어서 각각 요약(캡션) 생성
        response = openai_mm_llm.complete(
            prompt=prompt_template,
            image_documents=[img_node]
        )
        # ImageNode의 metadata에 요약 text를 저장하는('caption')을 추가
        img_node.metadata["caption"] = response.text
    
    # 메타데이터가 갱신된 ImageDocument 리스트 반환
    return image_nodes


In [ ]:
# 이미지 문서를 노드로 변환시키고 LLM을 통한 설명 요약을 캡션에 추가 
doc_to_nodes = docs_to_image_nodes(img_docs=image_documents) # 이미지 노드 변환
image_nodes = summarize_images_captioning(image_nodes=doc_to_nodes) # 이미지 노드에 캡셔닝 추가

In [ ]:
# 이미지 노드 첫 요소 출력
print(f'Type : {type(image_nodes[0])}\n')
print('Metadata :')
print(image_nodes[0].metadata)

In [ ]:
# Step3. ImageNode를 Document로 변환

'''
이미지 요약 텍스트를 추가한 Image Node를 Document 객체로 변환한다.
실제 이미지를 텍스트를 통해 검색될 수 있도록 객체를 변환하는 과정
이 때 image id를 추가해 원본 이미지를 참조할 수 있게 한다.
'''
from llama_index.core.schema import Document, NodeRelationship, RelatedNodeInfo
new_caption_docs = []
for img_node in image_nodes:
    caption = img_node.metadata["caption"]  # 앞 단계에서 넣어둔 캡션
    caption_doc = Document(
        text=caption,
        metadata={
            "source": img_node.metadata.get("source", ""),
            "is_image_caption": True, # 이미지 캡셔닝 대상임을 표시
            
            # 조회 편의용으로 저장
            "image_path": img_node.image_path,      # node의 image_path
            "image_node_id": img_node.id_,          # 조회 키
        },
    )
    # 관계로도 연결
    caption_doc.relationships[NodeRelationship.SOURCE] = RelatedNodeInfo(
        node_id=img_node.id_
    )
    new_caption_docs.append(caption_doc)

In [ ]:
new_caption_docs[0]  # 제대로 변환되었는지 첫 요소 확인하기(ImageNode -> Document로 제대로 변환되었는지 확인)

In [ ]:
# 논문 text document + 이미지 text 캡셔닝 Document + 이미지 Document
documents = text_documents+new_caption_docs+image_documents 
print(f'문서의 개수 : {len(documents)}')

### 4. 인덱스 생성 및 RAG 구현

In [ ]:
# Step1. 인덱스 생성 및 저장
# 신규 생성 (기존 경로가 이미 존재 할 경우 불러옴)
from qdrant_client import QdrantClient
client = QdrantClient(path="./index/ch05_3_4_pdf_to_index_db")

In [ ]:
from llama_index.vector_stores.qdrant import QdrantVectorStore

# 텍스트에 대한 벡터 스토어 구성
text_store= QdrantVectorStore(client=client,
                              collection_name="text_collection")
# 이미지에 대한 벡터 스토어 구성
image_store = QdrantVectorStore(client=client, 
                              collection_name="image_collection")

# stroage context를 통해 멀티모달 벡터 스토어 및 인덱스를 통합관리
from llama_index.core import StorageContext
storage_context = StorageContext.from_defaults(
    vector_store=text_store,
    image_store=image_store
)

In [ ]:
# 멀티 모달 인덱스 구성 및 저장
from llama_index.core.indices.multi_modal.base import MultiModalVectorStoreIndex
index = MultiModalVectorStoreIndex.from_documents(
    documents=documents, # 표/텍스트 문서+이미지 문서+이미지 캡셔닝 문서
    storage_context=storage_context,
)
index.storage_context.persist(persist_dir="./index/ch05_3_4_pdf_to_index_db")

In [ ]:
# 저장된 StorageContext 로드
from llama_index.core import load_index_from_storage

storage_context = StorageContext.from_defaults(
    persist_dir="./index/ch05_3_4_pdf_to_index_db",
    vector_store=text_store,
    image_store=image_store
)

# 저장된 인덱스 로드
index = load_index_from_storage(storage_context)

### 5. 유사도 검색 및 결과 출력(이미지 포함)

In [ ]:
# Step2. 유사도 검색 및 결과 출력(이미지 포함)
from PIL import Image
import matplotlib.pyplot as plt

# 이미지를 출력하는 함수 작성
def plot_images(image_paths):
    images_shown = 0
    plt.figure(figsize=(16, 9))
    for img_path in image_paths:
        img_path = img_path.lower() # 소문자화
        if ".jpg" in img_path:
            image = Image.open(img_path)

            plt.subplot(3, 3, images_shown + 1)
            plt.imshow(image)
            plt.xticks([])
            plt.yticks([])

            images_shown += 1
            if images_shown >= 9:
                break

In [ ]:
retriever_engine = index.as_retriever(
    similarity_top_k = 2, # 텍스트(Document) 노드 상위 2개 검색
    image_similarity_top_k = 2, # 이미지(ImageDocument) 노드 상위 2개 검색
)

In [ ]:
# 쿼리 입력 및 검색(이미지, 텍스트 조회)
nodes = retriever_engine.retrieve('Please tell me about Query Encoder') 

# 텍스트 요약 캡셔닝을 한 문서가 제대로 검색 출력되는지 확인
captioning_image_paths=[] # Document 중 이미지 캡셔닝 문서 Node
image_node_paths=[] # ImageDocument로 검색 된 Node
for node in nodes:
    print(node)
    if 'is_image_caption' in node.metadata: # 노드의 메타데이터에 is_image_caption 키가 있는 경우  
        captioning_image_paths.append(node.metadata['image_path'])
    if type(node.node) == ImageNode: # ImageDocument로 인해 생성 된 노드인 경우
        image_node_paths.append(node.metadata['file_path'])

In [ ]:
print('** 이미지 캡셔닝 문서로 검색 된 이미지 출력 *')
plot_images(image_paths=captioning_image_paths) # 이미지 plot

In [ ]:
print('** ImageDocument로 검색 된 노드 이미지 출력 *')
plot_images(image_paths=image_node_paths) # 이미지 plot

In [ ]:
# 노드의 타입 및 메타데이터 확인
for n in nodes:
    node = getattr(n, "node", n)   # NodeWithScore → node 꺼내기
    print(f"Type : {type(node)}")
    print(f"Metadata keys : {node.metadata.keys()}\n")

### 6. 쿼리 엔진 검색 시스템 구현
[참고] 이미지 캡셔닝을 통해 모든 imageDocument -> Document로 변경 후, Document로만 RAG 시스템을 구성시 OpenAIMultiModal 클래스를 사용하지 않고 
기존 일반 텍스트 처리 모델 OpenAI 를 사용해도 무관합니다. Document 객체에서 이미지에 대한 metadata 를 추가했기 때문입니다.
단, 이미지 문서인 ImageDocument를 같이 입력하는 멀티 모달 RAG를 같이 구현하는 경우에는 OpenAIMultiModal을 사용해야 합니다. 이번 실습에서는 ImageDocument와 Document가 섞여있는 구조로 OpenAIMultiModel을 사용합니다

In [ ]:
# Step3. 쿼리 엔진 구현
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
openai_mm_llm = OpenAIMultiModal(model="gpt-4o", 
                                 max_new_tokens=3000,)

query_engine = index.as_query_engine(
    llm=openai_mm_llm,
    similarity_top_k = 1, # Document 1개 반환
    image_similarity_top_k = 1 # ImageDocument 1개 반환
)

In [ ]:
from llama_index.core import PromptTemplate
# 프롬프트 템플릿 정의
template = (
    "제공 된 정보를 참고하여 한국어로 답변을 해주세요.\n"
    "{query_str}\n"
)
prompt_template = PromptTemplate(template)

In [ ]:
query_str = "Please tell me about Query Encoder"
final_prompt = prompt_template.format(query_str=query_str)
response = query_engine.query(final_prompt)

# prompt와 response 출력
print(f"Prompt:{final_prompt}")
print(f"Response:\n{response}")

In [ ]:
from llama_index.core.schema import ImageNode, TextNode
image_paths=[] # 이미지 경로 및 노드 타입 저장
for n in response.source_nodes:
    node = getattr(n, "node", n)   # NodeWithScore → node 꺼내기
    print(f"Node : \n{node}\nType : {type(node)}\nMeta data : \n{node.metadata}\n")
    if isinstance(node, TextNode):
        if 'is_image_caption' in node.metadata.keys():
            image_paths.append(('Text Node', node.metadata['image_path']))
    if isinstance(node, ImageNode):
        image_paths.append(('Image Node', node.metadata['file_path']))

In [ ]:
# 응답에 참고한 이미지 패스
for path in image_paths:
    print(path)